In [37]:
from elasticsearch import Elasticsearch
import google.generativeai as genai
import requests
import time
import json
import os

In [2]:
import minsearch

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [41]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in documents:
    es_client.index(index=index_name, document=doc)

In [52]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [76]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm_gemini(prompt, model='gemini-1.5-flash'):
    genai.configure(api_key=os.environ['API_KEY'])
    
    model = genai.GenerativeModel(model_name=model)
    response = model.generate_content(prompt)
    
    return response.text

In [72]:
def llm_hf(prompt, model="meta-llama/Meta-Llama-3-8B-Instruct"):
    
    API_URL = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

    payload = {
        "inputs": prompt,
    }

    while True:
        response = requests.post(API_URL, headers=headers, json=payload).json()
        try:
            if "is currently loading" in response["error"]:
                print(response["error"])
                time.sleep(5)
            else:
                break
        except:
            break
    
    return response[0]["generated_text"]

In [63]:
def ll_hf_qa(query, search_results, model):

    API_URL = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

    context = ""
    context_template = """
    QUESTION: {question}
    ANSWER: {text}
    """.strip()

    for r in search_results:
        context += context_template.format(question=r["question"], text=r["text"]) + "\n\n"

    payload = {
    	"inputs": {
    		"question": query,
    		"context": context
    	},
    }
    
    while True:
        response = requests.post(API_URL, headers=headers, json=payload).json()
        try:
            if "is currently loading" in response["error"]:
                print(response["error"])
                time.sleep(5)
            else:
                break
        except:
            break
    	
    return response

In [55]:
def rag(query, llm, model):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    if llm == "gemini":
        answer = llm_gemini(prompt, model)
    elif llm == "hf":
        answer = llm_hf(prompt, model)
    elif llm == "hf_qa":
        answer = ll_hf_qa(query, search_results, model)
    else:
        raise Exception("llm not configured")
    
    return answer

In [44]:
q = 'the course has already started, can I still enroll?'

In [45]:
rag(q, llm="gemini", model="gemini-1.5-flash")

'Yes, you can still join the course. You won’t be able to submit some of the homeworks, but you can still take part in the course. \nTo get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. \nThis means that if you join at the end of November and manage to work on two projects, you will still be eligible for a certificate. \n'

In [46]:
rag(q, llm="gemini", model="gemini-1.5-pro")

'Yes, you can still join the course after the start date. You may not be able to submit all the homework since some deadlines might have passed, but you can still participate in the course. \n'

In [47]:
rag(q, llm="gemini", model="gemini-1.0-pro")

'Yes, you can still enroll in the course. However, you will not be able to submit some of the homeworks. You will still be eligible to take part in the course and obtain a certificate if you submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline.'

In [77]:
rag(q, llm="hf", model="google/flan-t5-large")

Model google/flan-t5-large is currently loading
Model google/flan-t5-large is currently loading


'The course has already started. Can I still join it?'

In [78]:
rag(q, llm="hf", model="meta-llama/Meta-Llama-3-8B-Instruct")

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: the course has already started, can I still enroll?\n\nCONTEXT: \nquestion: The course has already started. Can I still join it?\nanswer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.\n\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nquestion:

In [79]:
rag(q, llm="hf", model="mistralai/Mistral-7B-Instruct-v0.3")

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: the course has already started, can I still enroll?\n\nCONTEXT: \nquestion: The course has already started. Can I still join it?\nanswer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.\n\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nquestion: Cou

In [56]:
rag(q, llm="hf_qa", model="deepset/deberta-v3-large-squad2")

{'score': 0.14685525000095367, 'start': 74, 'end': 79, 'answer': ' Yes,'}

In [57]:
rag(q, llm="hf_qa", model="deepset/roberta-base-squad2")

{'score': 0.058043383061885834,
 'start': 75,
 'end': 87,
 'answer': 'Yes, you can'}

In [58]:
rag(q, llm="hf_qa", model="deepset/tinyroberta-squad2")

Model deepset/tinyroberta-squad2 is currently loading
Model deepset/tinyroberta-squad2 is currently loading


{'score': 7.168572483351454e-05,
 'start': 1111,
 'end': 1131,
 'answer': 'Can I still graduate'}

In [ ]:
rag(q, llm="hf_qa", model="deepset/tinyroberta-squad2")